<a href="https://colab.research.google.com/github/dorbodwolf/huggingface_nlp_course/blob/main/course/en/chapter2/section2_pt.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Behind the pipeline (PyTorch)

Install the Transformers, Datasets, and Evaluate libraries to run this notebook.

In [7]:
!pip install datasets evaluate transformers[sentencepiece]

In [9]:
# 数据来源
# https://www.kaggle.com/datasets/abhi8923shriv/sentiment-analysis-dataset
my_sentences = [
        "So hot today =_= don`t like it and i hate my new timetable, having such a bad week",
        "you guys didn`t say hi or answer my questions yesterday but nice songs.",
        'feels sorry every time I`m printing out, I use like 200 new papers',
        'You seem nice, you`re generous and you know your stuff'
    ]

In [10]:
from transformers import pipeline

classifier = pipeline("sentiment-analysis")
classifier(
    my_sentences
)

No model was supplied, defaulted to distilbert-base-uncased-finetuned-sst-2-english and revision af0f99b (https://huggingface.co/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.


[{'label': 'NEGATIVE', 'score': 0.9992561936378479},
 {'label': 'POSITIVE', 'score': 0.9994785189628601},
 {'label': 'NEGATIVE', 'score': 0.9994438290596008},
 {'label': 'POSITIVE', 'score': 0.9998743534088135}]

In [11]:
from transformers import AutoTokenizer

checkpoint = "distilbert-base-uncased-finetuned-sst-2-english"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

In [12]:
raw_inputs = my_sentences
inputs = tokenizer(raw_inputs, padding=True, truncation=True, return_tensors="pt")
print(inputs)

{'input_ids': tensor([[  101,  2061,  2980,  2651,  1027,  1035,  1027,  2123,  1036,  1056,
          2066,  2009,  1998,  1045,  5223,  2026,  2047, 23839,  1010,  2383,
          2107,  1037,  2919,  2733,   102],
        [  101,  2017,  4364,  2134,  1036,  1056,  2360,  7632,  2030,  3437,
          2026,  3980,  7483,  2021,  3835,  2774,  1012,   102,     0,     0,
             0,     0,     0,     0,     0],
        [  101,  5683,  3374,  2296,  2051,  1045,  1036,  1049,  8021,  2041,
          1010,  1045,  2224,  2066,  3263,  2047,  4981,   102,     0,     0,
             0,     0,     0,     0,     0],
        [  101,  2017,  4025,  3835,  1010,  2017,  1036,  2128, 12382,  1998,
          2017,  2113,  2115,  4933,   102,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 

In [13]:
from transformers import AutoModel

checkpoint = "distilbert-base-uncased-finetuned-sst-2-english"
model = AutoModel.from_pretrained(checkpoint)

In [14]:
outputs = model(**inputs)
print(outputs.last_hidden_state.shape)

torch.Size([4, 25, 768])


In [15]:
from transformers import AutoModelForSequenceClassification

checkpoint = "distilbert-base-uncased-finetuned-sst-2-english"
model = AutoModelForSequenceClassification.from_pretrained(checkpoint)
outputs = model(**inputs)

In [16]:
print(outputs.logits.shape)

torch.Size([4, 2])


In [17]:
print(outputs.logits)

tensor([[ 3.9396, -3.2633],
        [-3.6266,  3.9317],
        [ 4.1018, -3.3921],
        [-4.2980,  4.6840]], grad_fn=<AddmmBackward0>)


In [18]:
import torch

predictions = torch.nn.functional.softmax(outputs.logits, dim=-1)
print(predictions)

tensor([[9.9926e-01, 7.4384e-04],
        [5.2148e-04, 9.9948e-01],
        [9.9944e-01, 5.5613e-04],
        [1.2563e-04, 9.9987e-01]], grad_fn=<SoftmaxBackward0>)


In [19]:
model.config.id2label

{0: 'NEGATIVE', 1: 'POSITIVE'}